
Peer-graded Assignment: Week 3 Part 1

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from itertools import repeat

In [2]:
website_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

In [3]:
soup = BeautifulSoup(website_url,'lxml')

In [4]:
# get the table
my_table = soup.find("table", { "class" : "wikitable sortable"})

In [5]:
# convert BeautifulSoup tags to string list
def convert_to_list(bs4row):
    list_bs4row = bs4row.findAll(["td","th"])
    return [bs4.get_text().strip() for bs4 in list_bs4row]

In [6]:
# get the table 
rows=my_table.findAll("tr")

# first row is header
header = convert_to_list(rows[0])
header

['Postcode', 'Borough', 'Neighbourhood']

In [7]:
# convert to list of list
my_data = [convert_to_list(r) for r in rows[1:]]

In [8]:
df = pd.DataFrame(my_data,columns=header)
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


Ignore cells with a borough that is Not assigned

In [9]:
df = df[~df['Borough'].isin(['Not assigned'])]
df.head(10)

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor
7,M7A,Downtown Toronto,Queen's Park
9,M9A,Queen's Park,Not assigned
10,M1B,Scarborough,Rouge
11,M1B,Scarborough,Malvern
13,M3B,North York,Don Mills North



If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough

In [10]:
df['Neighbourhood'] = df.apply(lambda row: row['Borough'] if row['Neighbourhood']=='Not assigned' else row['Neighbourhood'], axis=1)
df.head(10)

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor
7,M7A,Downtown Toronto,Queen's Park
9,M9A,Queen's Park,Queen's Park
10,M1B,Scarborough,Rouge
11,M1B,Scarborough,Malvern
13,M3B,North York,Don Mills North


In [11]:
df = df.groupby(['Postcode','Borough'])['Neighbourhood'].apply(','.join).reset_index()

In [12]:
df.shape

(103, 3)